In [ ]:
#
# Visualization of data statistic
# 
import numpy as np
import pandas as pd
import matplotlib.pyplot as mplot
from matplotlib.figure import Figure 
#from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import colors as mcolors
import colorsys
import seaborn as sbn

# Data frame: 
#   data: data in dictionary with labels X: 'xl', Y: 'yl';
#         data[{xl, yl}]: matrix of size #samples * #variables
#   tag:  dictionary of X & Y variable names (optional), shown as labels of data in the output
class Dataframe:
    def __init__(self, data, tag = None, xl='X', yl='Y'):
        self.nx = len(data[xl][0])
        self.ndata = len(data[xl])
        dy = data[yl]
        try:
            self.ny = len(data[yl][0])
        except:
            self.ny = 1
            dy = [[data[yl][i]] for i in range(len(data[yl]))]
        print("Dataframe: ndata / nx / ny is {:d} {:d} {:d}\n".format(self.ndata, self.nx, self.ny))
        if tag is None:
            xlist = list([xl+'-'+str(i) for i in range(self.nx)])
            ylist = list([yl+'-'+str(i) for i in range(self.ny)])
        else:
            xlist = list(tag[xl])
            ylist = list(tag[yl])
        self.data = pd.DataFrame(np.array(data[xl]).reshape(self.ndata, self.nx), 
                                 columns=xlist)
        print(len(dy))
        self.clist = xlist
        self.clist.extend(ylist)
        dy = np.array(dy).T
        for i in range(self.ny):
            #print(list(dy[0:len(dy)][i]))
            self.data.insert(self.nx+i, ylist[i], list(dy[i]))
        self.block = 1
        self.colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
        self.colorlist = [ name for name, color in self.colors.items()]
        
    # Calculate autocorrelations
    #   Return the block size, the autocorrelation 
    #   tol: maximum autocorrelation
    def autocorr(self, tol):
        assert(1.>tol>0.)
        at = 1.0
        block = 0
        while at > tol:
            if block > 0:
                block += int((at-tol)*5+1)
            else:
                block = 1
            at = 0.
            for i in range(self.nx):
                atn = pd.Series(self.data.iloc[:,i].to_dict()).autocorr(block)
                if atn > at: 
                    at = atn
        return block, at 
    
    # Block data with block size 'blk'                             
    def blockdata(self, blk):
        if blk > 1:
            self.data = pd.DataFrame(np.array([[self.data.iloc[i:i+blk,j].mean() for j in range(self.nx+self.ny)]
                                             for i in range(0, self.ndata, blk)]).reshape(int(self.ndata/blk), self.nx+self.ny), 
                                columns=self.clist)
        self.block = blk
        return
    
    # Plot histgrams
    #   crange: range of variables using pure digital position location in the data table
    #   out: output filename, None for no store of the output
    def hist(self, crange = None, out = None):
        try:
            cmn = int(crange[0])
        except:
            cmn = 0
        try:
            cmx = int(crange[1])
        except:
            cmx = self.nx+self.ny
        if out is not None:
            #Figure()
            #canves = FigureCanvasTkAgg(fig)
            #fig.set_canvas(canves)
            sfx = '.pdf'
            n = 1
            while True:
                try:
                    pf = open(out+'.hist'+sfx, 'r')
                except:
                    break
                pf.close()
                sfx = '.'+str(n)+'.pdf'
                n+=1
            pp = PdfPages(out+'.hist'+sfx)
        n = 1
        for i in range(cmn, cmx, 10):
            if out is not None:
                fig = mplot.figure()
            j=i
            if i+10<cmx:
                j=i+10
            else:
                j=cmx
            self.data.iloc[:,i:j].plot.hist(subplots=True, bins=int(self.ndata/4), title='Data(X Y) histgram')
            mplot.show()
            if out is not None:
                #fig.add_subplot(n, 1, n)
                pp.savefig(fig)
            n += 1 
        #if out is not None:   
            fig.savefig(pp, format = 'pdf')
        return
      
    # Density plot
    #   data: pandas.DataFrame; None for using stored data
    def density(self, crange = None, out = None, data=None):
        try:
            cmn = int(crange[0])
        except:
            cmn = 0
        try:
            cmx = int(crange[1])
        except:
            if data is None:
                cmx = self.nx+self.ny
            else:
                cmx = len(data.columns)
        if data is None:
            data = self.data
        if out is not None:
            fig = Figure()
            canves = FigureCanvasTkAgg(fig)
            fig.set_canvas(canves)
        n = 1
        for i in range(cmn, cmx, 10):
            j=i
            if i+10<cmx:
                j=i+10
            else:
                j=cmx
            data.iloc[:,i:j].plot.density()
            mplot.show()
            if out is not None:
                fig.add_subplot(n, 1, n)
            n += 1 
        if out is not None:
            sfx = '.pdf'
            n = 1
            while True:
                try:
                    pf = open(out+'.kde'+sfx, 'r')
                except:
                    break
                pf.close()
                sfx = '.'+str(n)+'.pdf'
                n+=1               
            fig.savefig(out+'.kde'+sfx, format = 'pdf')
        return
    
    # Box plot 
    #   title for the plot title
    #   data: pandas.DataFrame
    #   dform: 'pdf' for plotting according to p, z, (T)
    def boxplot(self, crange = None, data = None, title=None, out = None, dform = 'pdf'):
        def make_color(cols, pred):
            def lighten_color(color, amount=0.5):
                """
                Lightens the given color by multiplying (1-luminosity) by the given amount.
                Input can be matplotlib color string, hex string, or RGB tuple.

                Examples:
                >> lighten_color('g', 0.3)
                >> lighten_color('#F034A3', 0.6)
                >> lighten_color((.3,.55,.1), 0.5)
                """
                try:
                    c = mcolors.cnames[color]
                except:
                    c = color
                c = colorsys.rgb_to_hls(*mcolors.to_rgb(c))
                return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
            colors = mcolors.cnames #dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
            colorlist = [ name for name, color in colors.items()]
            tg={}
            for col in cols:
                if pred and ('pred' not in col):
                    continue
                if ('pred' in col) and (pred is False):
                    continue
                if col.split('+t+')[0] not in tg:
                    tg[col.split('+t+')[0]] = None
            i=0
            for col in tg:
                p=int(col.split('+')[0])
                z=col.split('+')[1]
                #t=int(col.split('+')[-2])
                #ri=col.split('+')[-1]
                if 'T' in col:
                    T = col.split('+')[-1]
                else:
                    T = str(None)
                clr = colorlist[i]
                if pred:
                    clr = lighten_color(clr, 0.5)
                tg[col] = clr
                i += 1
            return tg    
        #self.dform = dform
        try:
            cmn = int(crange[0])
        except:
            cmn = 0
        try:
            cmx = int(crange[1])
        except:
            if data is None:
                cmx = self.nx+self.ny
            else:
                cmx = len(data.columns)
        if data is None:
            data = self.data
        if False:#if out is not None:
            fig = Figure()
            canves = FigureCanvasTkAgg(fig)
            fig.set_canvas(canves) 
        n = 1
        if dform == 'pdf':
            #self.NA = str(None)
            if out is not None:
                sfx = '.pdf'
                m = 1
                while True:
                    try:
                        pf = open(out+'.box'+sfx, 'r')
                    except:
                        break
                    pf.close()
                    sfx = '.'+str(m)+'.pdf'
                    m+=1
                pltname = out+'.box'+sfx
                fig = mplot.figure()
            clrpred = make_color(data.columns, True)
            clr = make_color(data.columns, False)
            for col in clr:
                cln = []
                tl = []
                for c in data.columns:
                    if ('pred' not in c) and (col+'+t+' in c):
                        cln.append(c)
                        tl.append(int(c.split('+')[-2]))
                data.loc[:, cln].plot.box(title=title, colormap=clr[col], subplots=True, 
                                          sharex=True, sharey=True, xticks=tl)
                mplot.show()
                if out is not None:
                    fig.add_subplot()
                del cln
                del tl
            for col in clrpred:
                cln = []
                tl = []
                for c in data.columns:
                    if ('pred' in c) and (col+'+t+' in c):
                        cln.append(c)
                        tl.append(int(c.split('+')[-2]))
                data.loc[:, cln].plot.box(title=title, colormap=clrpred[col], subplots=True, 
                                          sharex=True, sharey=True, xticks=tl)
                mplot.show()
                if out is not None:
                    fig.add_subplot()
                del cln
                del tl
            if out is not None:
                fig.savefig(pltname, format='pdf')
            return   
        for i in range(cmn, cmx, 10):
            j=i+10
            if j>cmx:
                j=cmx
            data.iloc[:,i:j].plot.box(title=title)
            mplot.show()
            if out is not None:
                sfx = '.pdf'
                m = 1
                while True:
                    try:
                        pf = open(out+'.box'+sfx, 'r')
                    except:
                        break
                    pf.close()
                    sfx = '.'+str(m)+'.pdf'
                    m+=1               
                mplot.savefig(out+'.box'+sfx, format = 'pdf')
                #fig.add_subplot(n, 1, n)
            n += 1 
        if False:#if out is not None:
            sfx = '.pdf'
            n = 1
            while True:
                try:
                    pf = open(out+'.box'+sfx, 'r')
                except:
                    break
                pf.close()
                sfx = '.'+str(n)+'.pdf'
                n+=1               
            fig.savefig(out+'.box'+sfx, format = 'pdf')
        return
    
    # Errorbar plot; default data format 'pdf':
    #   data: dictionary of 'x', 'y', ('xerr', 'yerr')
    #         each item is the dictionary of data features in the arraylike object
    #   shape: specify plot layout: 
    #          None (default): all z's in a row, all p's in a column, all T's in same subplot
    #          'z': all z's in same subplot, all T's in a row
    #          'p': all p's in same subplot, all T's in a column
    def errbarplot(self, crange = None, data = None, shape = None, title=None, out = None, dform = 'pdf'):
        colorlist = [ name for name, color in mcolors.cnames.items()]
        def lighten_color(color, amount=0.8):
            """
            Lightens the given color by multiplying (1-luminosity) by the given amount.
            Input can be matplotlib color string, hex string, or RGB tuple.

            Examples:
            >> lighten_color('g', 0.3)
            >> lighten_color('#F034A3', 0.6)
            >> lighten_color((.3,.55,.1), 0.5)
            """
            try:
                c = mcolors.cnames[color]
            except:
                c = color
            c = colorsys.rgb_to_hls(*mcolors.to_rgb(c))
            return colorsys.hls_to_rgb(c[0], 1. - amount * (1. - c[1]), c[2])  
        def set_limit(data, derr):
            #emax = np.array(yerr).max()
            d = []
            err = []
            for key in data:
                try:
                    data[key].remove(np.NAN)
                    derr[key].remove(np.NAN)
                except:
                    pass
                d.extend(data[key])
                err.extend(derr[key])
            dmax = np.array(d).max()
            dmin = np.array(d).min()
            emax = np.array(err).max()
            sigma = 0.
            q = 1.0/float(len(d))
            qmax = 0.95
            bd = (1.+q)**len(d)
            i=1.
            llim = None
            ulim = None
            while 1.0-i*q > qmax:
                sigma = np.quantile(np.array(d), i*q) - np.quantile(np.array(d), (i-1.)*q)
                if sigma <= emax*bd:
                    break
                i += 1.0
            if True:#if i > 1.0:
                llim = np.quantile(np.array(d), (i-1.)*q)-emax
            i=1.
            while i*q < 1.-qmax:
                sigma = np.quantile(np.array(d), 1.-(i-1.)*q) - np.quantile(np.array(d), 1-i*q)
                if sigma <= emax*bd:
                    break
                i += 1.0
            if True:#if i > 1.0:
                ulim = np.quantile(np.array(d), 1-(i-1.)*q)+emax
            return llim, ulim
        try:
            cmn = int(crange[0])
        except:
            cmn = 0
        try:
            cmx = int(crange[1])
        except:
            if data is None:
                cmx = self.nx+self.ny
            else:
                cmx = len(data['x'].items())
        if data is None:
            data = self.data   
        if dform == 'pdf':
            pd_offset = 0.2
            #self.NA = str(None)
            if out is not None:
                sfx = '.pdf'
                m = 1
                while True:
                    try:
                        pf = open(out+'.errbar'+sfx, 'r')
                    except:
                        break
                    pf.close()
                    sfx = '.'+str(m)+'.pdf'
                    m+=1
                pltname = out+'.errbar'+sfx
                #fig = mplot.figure()
            zl=[]
            pl=[]
            Tl=[]
            for tg in data['x']:
                try:
                    p,z,T,pred=tg.split('+')
                except:
                    p,z,pred=tg.split('+')
                    T=str(None)
                if z not in zl:
                    #zl[z] = len(zl)
                    zl.append(z)
                if p not in pl:
                    #pl[p] = len(pl)
                    pl.append(p)
                if T not in Tl:
                    #Tl[T] = len(Tl)
                    Tl.append(T)
            zl.sort()
            pl.sort()
            Tl.sort()
            zd = {}
            pd = {}
            Td = {}
            for i in range(len(zl)):
                zd[zl[i]] = i
            for i in range(len(pl)):
                pd[pl[i]] = i
            for i in range(len(Tl)):
                Td[Tl[i]] = i
            if shape == 'p':
                row = len(zl)
                col = len(Tl)
                #posit = lambda: p,z,T(if(z==str(None) tl='spectrum'; return Td[T],zd[z])
                def posit(p,z,T):
                    if z==str(None):
                        tl = 'Spectrum'
                    elif T == str(None):
                        tl = 'DA z={:}'.format(z)
                    else:
                        tl = 'PDF z={:}, T={:}'.format(z,T)
                    return Td[T], zd[z], tl
            elif shape == 'z':
                row = len(Tl)
                col = len(pl)
                #posit = lambda: p,z,T(return pd[p],Td[T])
                def posit(p,z,T):
                    if z==str(None):
                        tl = 'Spectrum p={:}'.format(p)
                    elif T == str(None):
                        tl = 'DA p={:}'.format(p)
                    else:
                        tl = 'PDF p={:}'.format(p)
                    return pd[p], Td[T], tl
            else:
                row = len(zl)
                col = len(pl)
                #posit = lambda: p,z,T(return pd[p],zd[z])
                def posit(p,z,T):
                    if z==str(None):
                        tl = 'Spectrum p={:}'.format(p)
                    elif T == str(None):
                        tl = 'DA p={:} z={:}'.format(p, z)
                    else:
                        tl = 'PDF p={:} z={:}'.format(p, z)
                    return pd[p], zd[z], tl
            fig, ax = mplot.subplots(nrows=row, ncols=col, sharex=True, sharey=True)
            tgmark = []
            i = 0
            lylim, uylim = set_limit(data['y'], data['yerr']) 
            for tg in data['x']:
                try:
                    p,z,T,pred = tg.split('+')
                except:
                    p,z,pred = tg.split('+')
                    T = str(None)
                try:
                    yerr = data['yerr'][tg]
                except:
                    yerr = None
                try:
                    xerr = data['xerr'][tg]
                except:
                    xerr = None
                # generate color code using p, z, T
                ccode = int(p)+0X80
                if z != str(None):
                    ccode = (ccode<<8) + int(z)+0X80
                    if T != str(None):
                        ccode = (ccode<<8) + int(T)+0X80
                color = "#%06X" % ccode    #colorlist[i]
                if False: #if p+'+'+z+'+'+T not in tgmark:
                    tgmark.append(p+'+'+z+'+'+T)
                    i += 1
                if pred == 'pred':
                    color = lighten_color(color)
                    #data['x'][tg] += pd_offset
                c,r,tl = posit(p,z,T)
                if row==1 and col==1:
                    ax.set_ylim(ymin=lylim, ymax=uylim)
                    ax.errorbar(data['x'][tg], data['y'][tg], xerr=xerr, yerr=yerr, 
                                          color=color, barsabove=True, fmt='o', capthick=10)
                    ax.set_title(tl)
                elif row>1 and col>1:
                    ax[r,c].set_ylim(ymin=lylim, ymax=uylim)
                    ax[r,c].errorbar(data['x'][tg], data['y'][tg], xerr=xerr, yerr=yerr, 
                                          color=color, barsabove=True, fmt='o', capthick=10)
                    ax[r,c].set_title(tl)
                else:
                    ax[r+c].set_ylim(ymin=lylim, ymax=uylim)
                    ax[r+c].errorbar(data['x'][tg], data['y'][tg], xerr=xerr, yerr=yerr, 
                                          color=color, barsabove=True, fmt='o', capthick=10)
                    ax[r+c].set_title(tl)
            if title is not None:
                fig.suptitle(title)
            mplot.show()
            del pl
            del zl
            del Tl
            del pd
            del zd
            del Td
            if out is not None:
                fig.savefig(pltname, format='pdf')
        return
    
    
    # Error bar plot
    #   x / y / err: x labels, y values, y errors
    #   clr: color code
    def errbarplot1(x, y, err, clr = None, title=None, out=None, dform = 'pdf'):
        colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
        colorlist = [ name for name, color in colors.items()]
        try:
            errbarcolor = (clr+len(colorlist))%len(colorlist)
        except:
            errbarcolor = 0
        mplot.errorbar(x, y, yerr=err, ecolor=colorlist[errbarcolor], title=title)
        mplot.show()
        if out is not None:
            sfx = '.pdf'
            n = 1
            while True:
                try:
                    pf = open(out+'.err'+sfx, 'r')
                except:
                    break
                pf.close()
                sfx = '.'+str(n)+'.pdf'
                n+=1               
            mplot.savefig(out+'.err'+sfx, format = 'pdf')
        return
    
    # Plot covariance matrix
    #   prange: list of X & Y variable ranges of [min, max] using pure digital position location
    def covplot(self, prange = None, out = None):
        try:
            pmn = tuple(prange[0])
            for i in range(2):
                if pmn[i] is None:
                    pmn[i] = 0
        except:
            pmn = (0,0)
        try:
            pmx = tuple(prange[1])
            for i in range(2):
                if pmx[i] is None:
                    pmx[i] = self.nx+self.ny
        except:
            pmx = (self.nx+self.ny, self.nx+self.ny)
        corrl = self.data.corr().iloc[pmn[0]:pmx[0], pmn[1]:pmx[1]]
        print(corrl)
        #corrl.plot.hexbin(x=corrl[0], y=corrl[1])
        if out is not None:
            fig = Figure()
            canves = FigureCanvasTkAgg(fig)
            fig.set_canvas(canves)
        n = 1
        for j in range(0, pmx[1]-pmn[1], 16):
            jm = j+16
            if jm > pmx[1]-pmn[1]:
                jm = pmx[1]
            for i in range(0, pmx[0]-pmn[0], 16):
                im = i+16
                if im > pmx[0]-pmn[0]:
                    im = pmx[0]
                tcorr = corrl.iloc[i:im, j:jm]
                sbn.heatmap(tcorr, cmap=sbn.diverging_palette(220, 20, as_cmap=True))#, xticklabels=tcorr.columns, yticklabels=tcorr.index)
                mplot.show()
                if out is not None:
                    fig.add_subplot(n, 1, n)
        if out is not None:
            sfx = '.pdf'
            n = 1
            while True:
                try:
                    pf = open(out+'.corr'+sfx, 'r')
                except:
                    break
                pf.close()
                sfx = '.'+str(n)+'.pdf'
                n+=1               
            fig.savefig(out+'.corr'+sfx, format = 'pdf')
        return
    

    def make_color(self, cols, pred):
        def lighten_color(color, amount=0.5):
            """
            Lightens the given color by multiplying (1-luminosity) by the given amount.
            Input can be matplotlib color string, hex string, or RGB tuple.

            Examples:
            >> lighten_color('g', 0.3)
            >> lighten_color('#F034A3', 0.6)
            >> lighten_color((.3,.55,.1), 0.5)
            """
            try:
                c = mcolors.cnames[color]
            except:
                c = color
            c = colorsys.rgb_to_hls(*mcolors.to_rgb(c))
            return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
        colors = mcolors.cnames #dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
        colorlist = [ name for name, color in colors.items()]
        tg={}
        for col in cols:
            if pred and ('pred' not in col):
                continue
            if ('pred' in col) and (pred is False):
                continue
            if col.split('+t+')[0] not in tg:
                tg[col.split('+t+')[0]] = None
        i=0
        for col in tg:
            p=int(col.split('+')[0])
            z=col.split('+')[1]
            #t=int(col.split('+')[-2])
            #ri=col.split('+')[-1]
            if 'T' in col:
                T = col.split('+')[-1]
            else:
                T = str(None)
            clr = colorlist[i]
            if pred:
                clr = lighten_color(clr, 0.5)
            tg[col] = clr
            i += 1
        return tg    
    